In [1]:
import random
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LassoCV

random.seed(1)

In the following chunk of code, I am importing the initial cleaned dataset, but we will need to do additional cleaning. Steps that we will be taking:

1. Dropping the 'Unnamed: 0' column as it contains the original index of the cleaned dataset (which we do not need)
2. Filtering the dataset to only include the entries with successful ('1') or failed ('0') projects
3. Filtered the dataset to only use games as a main category subset for more specific analysis. Games was the category with the most backers

In [2]:
kickstarter_clean = pd.read_csv('kickstarter_cleaned_index.csv')
kickstarter_clean = kickstarter_clean.drop(columns = ['Unnamed: 0'])
#kickstarter_clean['year_launched'] = kickstarter_clean.launched.apply(lambda x:int(x[0:4]))

In [3]:
kickstarter_clean_fail_successful = kickstarter_clean[(kickstarter_clean.state == '1') | (kickstarter_clean.state == '0')] 

kickstarter_games = kickstarter_clean_fail_successful[kickstarter_clean_fail_successful['main_category'] == 'Games']

print(kickstarter_games.head())

            ID                                               name  \
10  1000057089  Tombstone: Old West tabletop game and miniatur...   
38  1000170964  Penny Bingo Playing Card Game fun for the whol...   
42  1000190427                                       Seas of Iron   
52  1000232288                                     Empires at Sea   
68  1000328150  Legacy of Svarog | a Unique 3D Action RPG and ...   

          category main_category currency             deadline     goal  \
10  Tabletop Games         Games      GBP  2017-05-03 00:00:00   5000.0   
38  Tabletop Games         Games      USD  2017-03-27 00:00:00   1500.0   
42  Tabletop Games         Games      USD  2013-07-19 00:00:00   3000.0   
52  Tabletop Games         Games      USD  2015-06-23 00:00:00  25000.0   
68     Video Games         Games      USD  2015-10-30 00:00:00  50000.0   

               launched state  backers country  usd_pledged_real  \
10  2017-04-05 19:44:18     1      761      GB         121857.33  

We will now split the dataset into training and testing. We will also convert the y of the train and test sets into integers to run our logistic regression using smf.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(kickstarter_games[['ID','name','category','main_category','currency','deadline','goal','launched','backers','country','usd_pledged_real','usd_goal_real','project_length','pledged_per_backer']],kickstarter_games[['state']], test_size=0.3)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

y_train['state'] = y_train.state.apply(int)
y_test['state'] = y_test.state.apply(int)

(18932, 14) (18932, 1)
(8115, 14) (8115, 1)


In [5]:
#SKLEARN STUFF
#logreg_model = LogisticRegression()
#VARIABLES WE WANT
#X_train_goal_length = X_train[['goal','project_length']]
#X_test_goal_length = X_test[['goal','project_length']]
#logitfit = smf.logit(formula = str(f), data = hgc).fit()
#logreg_model.fit(X_train_goal_length,y_train)
#print(logreg_model.coef_)
#print(logreg_model.intercept_)

We will now run our model using the X_train and y_train datasets.
We can observe from the model summary that Tabletop Games appear to have the best project success rates, while Mobile Games appear to have the least.

In [6]:
logitmodel = smf.logit(formula = 'state ~ category + usd_goal_real + project_length + pledged_per_backer', data = pd.concat([X_train, y_train], axis=1))
results = logitmodel.fit()
print(results.summary())

Optimization terminated successfully.
         Current function value: 0.590210
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                  state   No. Observations:                18932
Model:                          Logit   Df Residuals:                    18921
Method:                           MLE   Df Model:                           10
Date:                Sun, 24 Nov 2019   Pseudo R-squ.:                  0.1450
Time:                        00:25:13   Log-Likelihood:                -11174.
converged:                       True   LL-Null:                       -13068.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       0.2680      0.077      3.484      0.000   

We obtain an accuracy of ~70% on the test dataset. TO DO: Crossvalidation to verify our results (but based on running this code multiple times the 70% appears to hold steady).

In [7]:
testresult_logit = results.predict(X_test)
#print(wholedataset_log)
testresult_logit = np.where(testresult_logit > 0.5, 1, 0)
print(testresult_logit)
print(y_test.state.to_numpy())
np.mean(testresult_logit == y_test.state.to_numpy()) #~70% accuracy on test dataset


[1 0 0 ... 1 1 0]
[1 0 1 ... 1 1 0]


0.7043746149106592

The number of failed projects constitute 54% of our training data. Thus, our model does represent a noticeable improvement in prediction accuracy (compared to a simple 'failed' prediction for all the data)

In [8]:
y_test.state.value_counts().iloc[0]/(y_test.state.value_counts().iloc[0] + y_test.state.value_counts().iloc[1])

0.5356746765249538

Out of interest, I also checked the number of each subcategory within the 'Games' main category. Tabletop Games were indeed the most popular type of project on Kickstarter, perhaps as a result of the higher success rate of such projects on the website. We also observe that more successful projects have a higher amount pledged per backer, which was a different result from my hypothesis that small donations (but many backers) could lead to more successful projects.

In [9]:
print(kickstarter_games[['pledged_per_backer','state']].sort_values('pledged_per_backer').groupby('state').mean())

#plot.bar(x='state',y='pledged_per_backer')
#plt.show()
kickstarter_games.category.value_counts()

       pledged_per_backer
state                    
0               45.824228
1               64.125363


Tabletop Games     11657
Video Games         8858
Games               2307
Playing Cards       1946
Mobile Games        1116
Live Games           714
Gaming Hardware      260
Puzzles              189
Name: category, dtype: int64